In [1]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

from keras.layers import Dense, Embedding, Bidirectional, GRU
from numpy import array
import numpy as np
import codecs
from tqdm import tqdm
from keras.preprocessing.text import text_to_word_sequence
tqdm.pandas()

Using TensorFlow backend.


In [2]:
def load_embeddings(word_index, max_features):    
    embeddings_index = {}
    f = codecs.open('word_embeddings/w2v_300.txt', encoding='utf-8')
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('%s word vectors encontrados' % len(embeddings_index))
    
    words_not_found = []
    
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            # palavras nao encontradas no embedding permanecem com valor nulo
            embedding_matrix[i] = embedding_vector
        else:
            words_not_found.append(word)
    
    print('Quantidade de word embeddings nulas: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    
    return embedding_matrix

In [3]:
def clean_text(text):
    text = text.replace('"', ' ')
    text = text.replace('\'', ' ')    
    text = text.replace('<', ' ')
    text = text.replace('>', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('*', ' ')
    text = text.replace('\\', ' ')
    text = text.strip()
    return text.lower()

In [4]:
# gera uma sequência de palavras, a partir de uma sequência anterior
# nós usaremos esta função para testar nosso modelo
def generate_seq(model, tokenizer, max_length, in_text, n_words):

    for _ in range(n_words):
        # cria uma sequência de valores inteiros a partir do texto de entrada
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        
        # fixa o tamanho das sequências
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        
        # estima a probabilidade para cada palavra da sequência dada
        yhat = model.predict_classes(encoded, verbose=0)
        
        # esta palavra está presente no índice de palavras do seu vocabulário?
        # Se a palavra já foi observada, então vamos
        # incluí-la na frase que está sendo gerada.
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # concatena a palavra na sequência sendo gerada
        in_text += ' ' + out_word
    return in_text

In [5]:
data = []
with open('dataset.txt', mode='r', encoding='utf-8') as file:
    for line in file:
        data.append(clean_text(line))

In [6]:
data = "\n".join(data)

tokenizer = Tokenizer()

tokenizer.fit_on_texts([data])

vocab_size = len(tokenizer.word_index) + 1
print('Tamanho do vocabulário: %d' % vocab_size)

# cria uma sequência de valores inteiros a partir do texto de entrada
encoded = tokenizer.texts_to_sequences([data])[0]

# Aqui, cada palavra estimada está condicionada àquelas palavras
# que aparecem antes dela na sequência. Para estimar uma palavra específica,
# nós consideramos as 2 palavras que vêm antes dela.
sequences = list()
for i in range(2, len(encoded)):
    sequence = encoded[i - 2: i + 1]
    sequences.append(sequence)


print('Número de sequências: %d' % len(sequences))

# apenas para assegurar que teremos sequências sempre com um mesmo tamanho
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')

print('Tamanho máximo da sequência: %d' % max_length)

# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)


word_index = tokenizer.word_index
max_features = len(word_index) + 1

Tamanho do vocabulário: 8693
Número de sequências: 347761
Tamanho máximo da sequência: 3


In [7]:
# carrega nosso word embedding pré-treinado
embedding_matrix = load_embeddings(word_index, max_features)

623264it [01:52, 5564.40it/s]


623264 word vectors encontrados
Quantidade de word embeddings nulas: 52


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 300, weights=[embedding_matrix], input_length=max_length-1, trainable=False))
model.add(Bidirectional(GRU(50)))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

# treina o nosso modelo.
model.fit(X, y, epochs=32)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 300)            2607900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               105300    
_________________________________________________________________
dense_1 (Dense)              (None, 8693)              877993    
Total params: 3,591,193
Trainable params: 983,293
Non-trainable params: 2,607,900
_________________________________________________________________

Epoch 1/32
347761/347761 [==============================] - 171s 491us/step - loss: 3.4698
Epoch 2/32
347761/347761 [==============================] - 171s 493us/step - loss: 2.7337
Epoch 3/32
347761/347761 [==============================] - 176s 506us/step - loss: 2.5548
Epoch 4/32
347761/347761 [==============================] - 206s 592us/step - loss: 2.4543
Epo

In [ ]:
while True:
    sentence = input('\ninput> ')

    if sentence == 'exit':
        break

    print('\noutput:', generate_seq(model, tokenizer, max_length-1, sentence, 15))


input>  embargos de



output: embargos de declaração quando não preenchidos os requisitos previstos no art 896 § 1º a i da



input>  agravo de



output: agravo de instrumento em recurso de revista lei 13 015 2014 não observância dos requisitos de admissibilidade



input>  conhecido e



output: conhecido e provido agravo de instrumento em recurso de revista lei 13 015 2014 não observância dos



input>  o exame



output: o exame do recurso de revista lei 13 015 2014 não observância dos requisitos de admissibilidade do
